In [10]:
import os
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [11]:
def plot_graph(data, X, Y, title, plot_name, x_units="", y_units="", highlight_points=None):
    layout = go.Layout( xaxis=dict(title=f"{X} {x_units}"), yaxis=dict(title=f"{Y} {y_units}"), template='simple_white' )

    line_style = ['solid', 'dot', 'dash', 'longdash', 'dashdot', 'longdashdot']
    color=px.colors.qualitative.Plotly

    trace = []
    for idx, item in enumerate(data):
        trace.append(go.Scatter(x=item[X], y=(item[Y]), name=plot_name[idx], mode='lines', line=dict(color=color[idx%6], dash=line_style[idx%6])))

        if highlight_points:
            for point in highlight_points[idx]:
                trace.append(go.Scatter(
                    x=[point[X]],
                    y=[point[Y]],
                    mode='markers',
                    marker=dict(size=12, color=color[idx], symbol='diamond'),  # Customize marker appearance
                    showlegend=False
                ))

    fig = go.Figure(data=trace, layout=layout)

    fig.write_image(f"./plots/{title}.png",scale=4, width=1200, height=600)

    print(f"\nGenerated plot for {title}")
    fig.show()

In [12]:
vanilla = []
dtrack = []
vanilla_highlight_points = []
dtrack_highlight_points = []
for migration_type in os.listdir('.\\data\\Application-Overhead\\Quicksort'):
    for filename in os.listdir(f'.\\data\\Application-Overhead\\Quicksort\\{migration_type}'):
        start_second, start_sorts, end_second, end_sorts = 0, 0, 0, 0
        if filename.endswith('.csv'):
            with open(f'.\\data\\Application-Overhead\\Quicksort\\{migration_type}\\{filename}', 'r') as f:
                data = f.readlines()
                for line in data:
                    if 'START' in line:
                        start_second = int(line.split(',')[1])
                        start_sorts = int(line.split(',')[2])
                    elif 'COMPLETE' in line:
                        end_second = int(line.split(',')[1])
                        end_sorts = int(line.split(',')[2])

            size = filename.split('.')[0].strip('ram')
            if 'Dtrack' in filename:
                dtrack_highlight_points.append([{'Second': start_second, 'Number of Sorts': start_sorts}, {'Second': end_second, 'Number of Sorts': end_sorts}])
                dtrack.append([f'{size}', pd.read_csv(f'.\\data\\Application-Overhead\\Quicksort\\{migration_type}\\{filename}')])
            else:
                vanilla_highlight_points.append([{'Second': start_second, 'Number of Sorts': start_sorts}, {'Second': end_second, 'Number of Sorts': end_sorts}])
                vanilla.append([f'{size}', pd.read_csv(f'.\\data\\Application-Overhead\\Quicksort\\{migration_type}\\{filename}')])

for index, values in enumerate(vanilla):
    plot_graph([dtrack[index][1], vanilla[index][1]], 'Second', 'Number of Sorts', f'Application-Overhead.CPU.RAM-{vanilla[index][0]}', ['Dtrack', 'Vanilla'], '(s)', highlight_points=[dtrack_highlight_points[index], vanilla_highlight_points[index]])


Generated plot for Application-Overhead.CPU.RAM-Quicksort.Summary


In [ ]:
# summary = pd.read_csv('.\\data\\Application-Overhead\\Quicksort\\summary.csv')
# plot_graph([summary], 'RAM', 'Performance Reduction', 'Application-Overhead.CPU.RAM-Quicksort.Summary', ['Performance Reduction'], 'Size (GB)', '(%)')